# **Applied Deep Learning and Artificial Intelligence - Group Assignment 4**

**Group members:**
Sebastian Vinther Jensen


#**Description of assignment**

I want to make a analysis on master's degrees with the highest earning potential:
*  various fields

The model will be built using appropriate data analysis and machine learning techniques. After the model has been built, we will test and evaluate three different evaluation metrics to make the model as accurate as possible.
**bold text**

# **Loading libraries and packages**

In [1]:
!pip install --upgrade pip
!pip install gradio==3.50.2
!pip install accelerate --q
!pip install "pydantic>=1.9,<2.0"
!pip install pypdf --q
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq tokenizers==0.14.0 --progress-bar off
!pip install -qqq optimum==1.13.1 --progress-bar off
!pip install -qqq auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!pip install -qqq unstructured==0.10.16 --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

# **Loading data**

In [2]:
# Cloning Github repository to Google Files
!git clone https://github.com/sebvinther/final-assignment

Cloning into 'final-assignment'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [9]:
from langchain.document_loaders import PyPDFDirectoryLoader
# Loading the PDF files from Google Files
loader = PyPDFDirectoryLoader("/contentfinal-assignment/blob/main/Education_Guide_10.pdf")
docs = loader.load()
# Printing the number of pages of the loaded data
len(docs)

0

# **Data preprocessing**

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Splitting the text in smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

0

## **Embeddings**

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

# Creating embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

query_result = embeddings.embed_query(texts[0].page_content)
print(len(query_result))

IndexError: list index out of range

# **Integrating vector database**

In [6]:
from langchain.vectorstores import Chroma

# Saving the embeddings in the Chroma database
db = Chroma.from_documents(texts, embeddings, persist_directory="db")
results = db.similarity_search("Transformer models", k=2)
print(results[0].page_content)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

# **Loading model and setting configurations**

In [ ]:
# Loading the transformer model
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

# Create a configuration for text generation based on the specified model name
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Set the maximum number of new tokens in the generated text to 1024.
# This limits the length of the generated output to 1024 tokens.
generation_config.max_new_tokens = 1024

# Set the temperature for text generation. Lower values (e.g., 0.0001) make output more deterministic, following likely predictions.
# Higher values make the output more random.
generation_config.temperature = 0.0001

# Set the top-p sampling value. A value of 0.95 means focusing on the most likely words that make up 95% of the probability distribution.
generation_config.top_p = 0.95

# Enable text sampling. When set to True, the model randomly selects words based on their probabilities, introducing randomness.
generation_config.do_sample = True

# Set the repetition penalty. A value of 1.15 discourages the model from repeating the same words or phrases too frequently in the output.
generation_config.repetition_penalty = 1.15


# Create a text generation pipeline using the initialized model, tokenizer, and generation configuration
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

# Create a LangChain pipeline that wraps the text generation pipeline and set a specific temperature for generation
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

# **Creating QA system**

## **First prompt**

In [ ]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from textwrap import fill

template_general_1 = """
<s>[INST] As a career advisor, your goal is to provide guidance on areas of study and career paths that match an individual's interests and skills. Keep your responses concise, within 50 words.

{context}
{question}
[/INST]</s>
"""

prompt_general_1 = PromptTemplate(template=template_general_1, input_variables=["context", "question"])



In [ ]:
result = qa_chain(
    "how do i find out what masters degree earns the most money"
)
print(fill(result["result"].strip(), width=80))

In [ ]:
result = qa_chain(
    "i like working with numbers, what masters degree should i choose?"
)
print(fill(result["result"].strip(), width=80))

## **N-shot Prompting**

Improving the prompt with N-shot Prompting:

In [ ]:
template_general_2 = """
<s>[INST] As a career advisor, guide individuals on selecting fields of study and career directions by considering their preferences, strengths, and aspirations. Include examples for clarity and ensure your advice is succinct, within 50 words.

N-shot Learning Examples:
Q: How can I find the right career path for me?
A: Reflect on your interests, values, and skills. Consider career assessment tests for insights and research various industries and roles to find a match. Networking and informational interviews can also provide valuable perspectives.

{context}
Now, provide guidance based on the input provided:
{question}
[/INST]</s>
"""

prompt_general_2 = PromptTemplate(template=template_general_2, input_variables=["context", "question"])




qa_chain_2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_2},
)

In [ ]:
result_2 = qa_chain_2(
    "How do I find out what masters degree I want to study"
)
print(fill(result_2["result"].strip(), width=80))

In [ ]:
result_2 = qa_chain_2(
    "I liked macro economics and organisation"
)
print(fill(result_2["result"].strip(), width=80))

In [ ]:
template_general_3 = """
<s>[INST] Offer advice as a career advisor on choosing suitable study areas or professions, taking into account individual preferences and market demands. Your responses should be brief, within 50 words.

Generated knowledge prompting examples:
Q: What should I study to align with future job markets?
A: Explore fields with high growth potential, such as technology, healthcare, and renewable energy. Stay informed about industry trends and consider careers that match your skills and interests, offering both personal fulfillment and job security.

{context}
Respond to the following based on the guidelines provided:
{question}
[/INST]</s>
"""




prompt_3 = PromptTemplate(template=template_3, input_variables=["context", "question"])


qa_chain_3 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_3},
)


In [ ]:
result_3 = qa_chain_3(
    "How do I find out what masters degree I want to study"
)
print(fill(result_3["result"].strip(), width=80))

In [ ]:
result_3 = qa_chain_3(
    "I liked statistics and applied mathematics"
)
print(fill(result_3["result"].strip(), width=80))

In [ ]:
result_3 = qa_chain_3(
    "I liked macro economics and organisation"
)
print(fill(result_3["result"].strip(), width=80))

## **Final Instruction Prompt**
Combining N-shot and Generated Knowledge Prompting to make our final instruction prompt

In [ ]:
template_general_4 = """
<s>[INST] You are guiding on selecting fields of study and careers, considering personal interest and future trends. Offer concise advice, within 50 words.

N-shot and Generated knowledge prompting examples:
Q: How do I choose a field of study?
A: Assess your passions and how they might translate into a career. Research fields with robust growth forecasts, and consider how they align with your skills and interests. Seek advice from professionals in the field for real-world insights.

{context}
Please advise based on the input:
{question}
[/INST]</s>
"""

prompt_general_4 = PromptTemplate(template=template_general_4, input_variables=["context", "question"])




qa_chain_4 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_4},
)

In [ ]:
result_4 = qa_chain_4(
    "How do I find out what masters degree I want to study"
)
print(fill(result_4["result"].strip(), width=80))

In [ ]:
result_4 = qa_chain_4(
    "I liked macro economics and organisation"
)
print(fill(result_4["result"].strip(), width=80))

# **Gradio Interface**

In [ ]:
import gradio as gr

In [ ]:
import time

In [ ]:
bot_name = "Supervisor"

with gr.Blocks() as demo:
    gr.Markdown("### Master's Degree Program Advisor")
    gr.Markdown("I can help you find the program that's right for you. Ask me any question related to choosing a master's program.")

    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def reply_bot_1(message, chat_history):
      bot_result = qa_chain_4(message)
      chat_history.append((message, (bot_result["result"].strip()))),
      time.sleep(2),
      return "", chat_history

    msg.submit(reply_bot_1, [msg, chatbot], [msg, chatbot])

demo.queue().launch(share=True)